In [ ]:
%matplotlib inline
import re
import pandas as pd

# Open the doc

In [ ]:
with open('VA_My_HealtheVet_Blue_Button_Sample_Version_12_10.txt') as f:
    content = f.readlines()

In [ ]:
full_doc = "".join(content)

# Vaccination history

In [ ]:
search_string = """
Immunization:(.*)?\n
"""

In [ ]:
search_string = "Immunization:\s+(.*?)\s+.*?Date Received:\s+(.*?)$"

In [ ]:
results = re.findall(search_string, full_doc, re.DOTALL | re.MULTILINE)

In [ ]:
results

In [ ]:
vax_df = pd.DataFrame(results, columns = ['vaccine', 'vax_date'])

In [ ]:
vax_df

In [ ]:
def strip_extra(s):
    if "@" in s:
        cutoff = s.index("@")
        s = s[:cutoff]
    return(s)

In [ ]:
vax_df['date'] = vax_df.vax_date.apply(lambda x: pd.Timestamp(strip_extra(x)))

In [ ]:
vax_df

In [ ]:
vax_df['clinically_documented'] = vax_df.vax_date.apply(lambda x: "@" in x)

In [ ]:
vax_df

In [ ]:
vax_df.sort_values('date', inplace = True)

In [ ]:
vax_df

In [ ]:
vax_df['vaccine_matching'] = vax_df.vaccine.apply(lambda x: x[:6].upper())

In [ ]:
vax_df

In [ ]:
vax_df.set_index('vaccine_matching')

In [ ]:
vax_df.groupby("vaccine_matching").date.apply(lambda x: [l.days for l in list(x - x.shift())[1:]])

# Medication reminders

In [ ]:
search_string = "Medication:\s+(.*?)\n+.*?Instructions:\s+(.*?)\n+.*?Status:\s+(.*?)\n+.*?Refills Remaining:\s+(.*?)\nLast Filled On:\s+(.*?)\nInitially Ordered On:\s+(.*?)\nQuantity:\s+(.*?)\nDays Supply:\s+(.*?)$"
results = re.findall(search_string, full_doc, re.MULTILINE)

In [ ]:
results

In [ ]:
medications = pd.DataFrame(results, columns = ['medication', 'directions', 'status', 'refills_remaining', 'last_filled_on', 'initially_ordered_on', 'quantity', 'days_supply'])
medications = medications.apply(lambda x: x.astype(str).str.upper())

In [ ]:
medications.head(3)

In [ ]:
medications[medications.status == 'ACTIVE']

In [ ]:
re.findall("EVERY\s(.*)", "TAKE ONE TABLET BY MOUTH EVERY DAY")[0]

In [ ]:
medications['time_delta'] = medications.directions.apply(lambda x: re.findall("EVERY\s(.*)", x))

In [ ]:
medications[medications.status == 'ACTIVE'][['directions', 'time_delta']]

In [ ]:
pd.date_range('now', periods = 30, frequency = 'D')

In [ ]:
pd.date_range('now', freq="12H", periods=10)